# Viriational Method for Hydrogen Ground State (SymPy Basic)

In this note, we will use the variational method to estimate the ground state energy of the hydrogen atom. And use ``SymPy`` to do the calculation. The trial wavefunction is chosen as
$$
\psi(\vec r) = (\frac{2a}{\pi})^{3/4} e^{-ar^2}
$$
which is normalized to 1 (in atomic units).

In [1]:
import sympy as sp
from sympy import exp, sqrt, pi, oo, integrate

# define variables and parameters
r, a = sp.symbols('r a', real=True, positive=True)

# ansatz wavefunction
psi = sqrt(sqrt(2*a/pi)**3) * exp(-a*r**2)

And the Hamiltonian of the hydrogen atom in atomic units is
$$
H = -\frac{1}{2} \nabla^2 - \frac{1}{r}
$$
where the Laplacian operator is
$$
\nabla^2 = \frac{1}{r^2} \frac{\partial}{\partial r} (r^2 \frac{\partial}{\partial r})
$$
for the spherically symmetric wavefunctions.

In [2]:
def apply_hamiltonion(psi):
    return -1/2 * psi.diff(r, 2) - 1/r * psi.diff(r) - (1/r) * psi

As verification, let's take a look at $\hat H \psi$.

In [3]:
sp.simplify(apply_hamiltonion(psi)/psi)

-2.0*a**2*r**2 + 3.0*a - 1/r

Now we can calculate the expectation value of the energy
$$
E = \frac{\langle \psi | \hat H | \psi \rangle}{\langle \psi | \psi \rangle}
$$

In [4]:
E = integrate(4*pi*r**2 * psi * apply_hamiltonion(psi), (r, 0, oo))
E

-2*sqrt(2)*sqrt(a)/sqrt(pi) + 1.5*a

To make use of more symbols, we denote the energy as
$$
E_a = - b \sqrt{a} + c a
$$
where $b=\frac{2\sqrt{2}}{\sqrt{\pi}}$ and $c=\frac{3}{2}$.

In [5]:
b, c = sp.symbols('b c', real=True, positive=True)
b_, c_ = 2*sqrt(2)/sqrt(pi), 3/2
Ea = - b * sqrt(a) + c * a

Tips for calculation by hand: there are two commonly appeared integrals,
$$
\int_0^\infty r^2 e^{-r^2} dr = \frac{\sqrt{\pi}}{4},\ \ 
\int_0^\infty r^4 e^{-r^2} dr = \frac{3\sqrt{\pi}}{8}
$$

In [6]:
am = sp.solve(Ea.diff(a), a)[0]
am

b**2/(4*c**2)

In [7]:
am_ = am.subs({b: b_, c: c_}).evalf()
am_

0.282942121052258

Or less symbolically,

In [8]:
sp.solve(E.diff(a), a)[0]

0.282942121052258

The energy is

In [9]:
E.subs(a, am).subs({b: b_, c: c_}).evalf()

-0.424413181578388

which is already very close to -1/2.

Finally, we can check the overlap between the trial wavefunction and the exact ground state wavefunction of the hydrogen atom, which is
$$
\psi_{\text{1s}}(\vec r) = \frac{1}{\sqrt{\pi}} e^{-r}
$$

In [10]:
psi_1s = 1/sqrt(pi) * exp(-r)
int_=integrate(4*pi*r**2*psi.subs(a, am_)*psi_1s, (r, 0, oo)).evalf()

int_

0.978179507402860

Note that the overlap integral cannot be analytically integrated, but can be expressed as summation of Gauss error functions. Thus it can be evaluated by SymPy.